In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

# Data loading

In [2]:
train_df = pd.read_csv("Train.csv")
test_df = pd.read_csv("Test.csv")
sample_submission = pd.read_csv('SampleSubmission.csv')

# Data preparation

In [3]:
# Split inputs and targets
train_inputs = train_df.drop(columns=['target'])
train_targets = train_df['target']
test_inputs = test_df.copy()

# Select features
dropped_columns = ['device', 'id']
train_inputs.drop(columns=dropped_columns, inplace=True)
test_inputs.drop(columns=dropped_columns, inplace=True)

# Transform categorical features
categorical_features = ['area']
oe = OrdinalEncoder()
train_inputs[categorical_features] = oe.fit_transform(train_inputs[categorical_features])
test_inputs[categorical_features] = oe.transform(test_inputs[categorical_features])

# Missing value imputation
train_inputs.fillna(0, inplace=True)
test_inputs.fillna(0, inplace=True)

### Prediction using XGBoost

In [4]:
X_train_xg, X_test_xg, y_train_xg, y_test_xg = train_test_split(
        train_inputs, train_targets, test_size=0.2, random_state=42
    )
model = XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        random_state=42
    )

model.fit(X_train_xg, y_train_xg)

y_pred_xg = model.predict(X_test_xg)

r2 = r2_score(y_test_xg, y_pred_xg)
rmse = np.sqrt(mean_squared_error(y_test_xg, y_pred_xg))
mae = mean_absolute_error(y_test_xg, y_pred_xg)



print(f"R²: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")


R²: 0.9417
RMSE: 10622065.0936
MAE: 6736943.0043


# Prediction algorithm using Random Forest

In [5]:
# Split training and validation tests

X_train, X_test, y_train, y_test = train_test_split(train_inputs, train_targets, test_size = 0.2, random_state = 0)

# Create algorithm
rf = RandomForestRegressor()

# Train
rf.fit(X_train, y_train)

# Validation
val_predictions = rf.predict(X_test)

# Test
test_predictions = rf.predict(test_inputs)

# Compute error metric

In [6]:
rmse = mean_squared_error(y_test, val_predictions, squared=False)
print(f"Root Mean Squared Error = {rmse / 1e6:.3} Mbit/s")
r2 = r2_score(y_test, val_predictions)
print(f"R²: {r2:.4f}")

Root Mean Squared Error = 9.78 Mbit/s
R²: 0.9491


# Save results

In [7]:
# Add index to results
predictions_df = pd.DataFrame({'id': test_df.id, 'target': test_predictions})
predictions_df.to_csv("BenchmarkSubmission.csv", index = False)
predictions_df.head()

,id,target
0,Id_ln0e0hfrgx,22375000.0
1,Id_svf7nz9fxv,62965000.0
2,Id_ww2mh07gwj,79366000.0
3,Id_v88r4y03ww,20463900.0
4,Id_2u4y4kzglh,3212600.0
